Training a model using the nnUnet library 
see https://github.com/MIC-DKFZ/nnUNet/tree/master

1 - Convert a subset the competition dataset to nnunet format
2 - 

In [2]:
subset_name = "CT/CT_Abdomen1K" # using a subset for now
nnunet_dataset_name = "Dataset003_CT_Abdomen1K"

In [3]:
from src.config import config
import os
train_dir = os.path.join(config["DATA_DIR"], "3D_train_npz_random_10percent_16G", subset_name)
nnunet_train_dir = os.path.join(config["DATA_DIR"], "nnunet_raw", nnunet_dataset_name)
os.makedirs(nnunet_train_dir, exist_ok=True)
os.makedirs(os.path.join(nnunet_train_dir, "imagesTr"), exist_ok=True)
os.makedirs(os.path.join(nnunet_train_dir, "imagesTs"), exist_ok=True)
os.makedirs(os.path.join(nnunet_train_dir, "labelsTr"), exist_ok=True)
os.makedirs(os.path.join(nnunet_train_dir, "labelsTs"), exist_ok=True)


In [63]:
import numpy as np
import nibabel as nib

files = os.listdir(train_dir)
for i, file in enumerate(files):
    subset = "Tr" if i < len(files) * 0.8 else "Ts"
    print(f"file : {file}, {i}/{len(files)}")
    data = np.load(os.path.join(train_dir, file), allow_pickle=True)
    # print(data.keys()) # imgs, gts, spacing
    imgs = data["imgs"]
    # add a channel dimension to the imgs
    #imgs = np.expand_dims(imgs, axis=0)
    affine = np.eye(4)
    affine[0, 0] = data["spacing"][0]
    affine[1, 1] = data["spacing"][1]
    affine[2, 2] = data["spacing"][2]
    nib_img = nib.Nifti1Image(imgs, affine=affine)
    nib.save(nib_img, os.path.join(nnunet_train_dir, "images"+subset, file.replace(".npz", "_0000.nii.gz")))

    gts = data["gts"]
    # only keep the label 1 for now
    gts = (gts == 1).astype(np.int8)
    nib_gt = nib.Nifti1Image(gts, affine=affine)
    nib.save(nib_gt, os.path.join(nnunet_train_dir, "labels"+subset, file.replace(".npz", ".nii.gz")))
    print(imgs.shape, gts.shape, spacing)
    




file : CT_Abdomen1K_Case_00813.npz, 0/100
(76, 512, 512) (76, 512, 512) [0.703125 0.703125 2.5     ]
file : CT_Abdomen1K_Case_00234.npz, 1/100
(66, 512, 512) (66, 512, 512) [0.703125 0.703125 2.5     ]
file : CT_Abdomen1K_Case_00151.npz, 2/100
(70, 512, 512) (70, 512, 512) [0.703125 0.703125 2.5     ]
file : CT_Abdomen1K_Case_00414.npz, 3/100
(409, 512, 512) (409, 512, 512) [0.703125 0.703125 2.5     ]
file : CT_Abdomen1K_Case_00350.npz, 4/100
(85, 512, 512) (85, 512, 512) [0.703125 0.703125 2.5     ]
file : CT_Abdomen1K_Case_00047.npz, 5/100
(225, 512, 512) (225, 512, 512) [0.703125 0.703125 2.5     ]
file : CT_Abdomen1K_Case_00977.npz, 6/100
(69, 512, 512) (69, 512, 512) [0.703125 0.703125 2.5     ]
file : CT_Abdomen1K_Case_00164.npz, 7/100
(37, 512, 512) (37, 512, 512) [0.703125 0.703125 2.5     ]
file : CT_Abdomen1K_Case_00612.npz, 8/100
(68, 512, 512) (68, 512, 512) [0.703125 0.703125 2.5     ]
file : CT_Abdomen1K_Case_00882.npz, 9/100
(40, 512, 512) (40, 512, 512) [0.703125 0.703

In [5]:
len(os.listdir(os.path.join(nnunet_train_dir,"imagesTr")))
    

80

In [8]:
#export the variables for dataset location (see https://github.com/MIC-DKFZ/nnUNet/blob/master/documentation/set_environment_variables.md)

os.environ["nnUNet_raw"] = os.path.join(config["DATA_DIR"], "nnunet_raw")
os.environ["nnUNet_preprocessed"] = os.path.join(config["DATA_DIR"], "nnUNet_preprocessed") 
os.environ["nnUNet_results"] = os.path.join(config["DATA_DIR"], "nnUNet_results")

In [48]:
# easy paste when using a terminal
for var_name in ["nnUNet_raw", "nnUNet_preprocessed", "nnUNet_results"]:
    print("export " + var_name + "=" + os.environ[var_name])

export nnUNet_raw=/nfs/data/nii/data1/Analysis/GPUnet/ANALYSIS_incontext/SegFM3D/data/nnunet_raw
export nnUNet_preprocessed=/nfs/data/nii/data1/Analysis/GPUnet/ANALYSIS_incontext/SegFM3D/data/nnUNet_preprocessed
export nnUNet_results=/nfs/data/nii/data1/Analysis/GPUnet/ANALYSIS_incontext/SegFM3D/data/nnUNet_results


In [ ]:
# Ressource request on nora

--time=2-0 --nodelist=loki --gpus-per-node=2  --cpus-per-task=16 --mem=64G

source /software/anaconda3/etc/profile.d/conda.sh
conda activate segfm3d_2
cd /nfs/norasys/notebooks/camaret/segfm3d_nora_team
/software/inetaccess/inetaccess camaret

export TORCH_COMPILE_DISABLE=1

using nninteractive plans :

In [ ]:
# cp nnInteractive_v1.0/plans.json $nnUNet_preprocessed/DatasetXXX_XXX/nnUNetResEncUNetLPlans_noResampling.json

# add a 3d_fullres_ps192_bs1 that sets batch size = 1 since we are gpu poor 

# preprocess the dataset
nnUNetv2_preprocess -d 2 -np 12 -plans_name nnUNetResEncUNetLPlans_noResampling -c 3d_fullres_ps192_bs1


# define a custom trainer in  nnUNet/nnunetv2/training/nnUNetTrainer/CustomTrainer.py
# train using the custom trainer with our preprocessing steps
nnUNetv2_train 2 3d_fullres_ps192 0 -p nnUNetResEncUNetLPlans_noResampling -tr CustomTrainer --num_gpus 1

In [10]:
# cp nnInteractive_v1.0/inference_session_class.json to $nnUNet_results/DatasetXXX_XXX/CustomTrainer__nnUNetResEncUNetLPlans_noResampling__3d_fullres_ps192_bs1

# weights are ready to be picked by the nnInteractive.inference.inference_session class
import torch
from nnInteractive.inference.inference_session import nnInteractiveInferenceSession
session = nnInteractiveInferenceSession(
    device=torch.device("cuda:0"),  # Set inference device
    use_torch_compile=False,  # Experimental: Not tested yet
    verbose=False,
    torch_n_threads=os.cpu_count(),  # Use available CPU cores
    do_autozoom=True,  # Enables AutoZoom for better patching
    use_pinned_memory=True,  # Optimizes GPU memory transfers
)

# Load the trained model
model_path = os.path.join(os.environ["nnUNet_results"], "Dataset002_CT_Abdomen1K/CustomTrainer__nnUNetResEncUNetLPlans_noResampling__3d_fullres_ps192_bs1")
session.initialize_from_trained_model_folder(model_path)


Building network architecture with 8 input channels


In [17]:
config

{'DATA_DIR': '/nfs/data/nii/data1/Analysis/GPUnet/ANALYSIS_incontext/SegFM3D/data',
 'RESULTS_DIR': '/nfs/data/nii/data1/Analysis/GPUnet/ANALYSIS_incontext/SegFM3D/results',
 'SAM_CKPT_PATH': '/nfs/norasys/notebooks/camaret/SAM-Med3D/ckpt/sam_med3d_turbo_bbox_cvpr.pth',
 'SAM_REPO_DIR': '/nfs/norasys/notebooks/camaret/SAM-Med3D',
 'ONNX_MODEL_PATH': '/nfs/norasys/notebooks/camaret/model_inference/models/sammed3d.onnx',
 'NNINT_CKPT_DIR': '/nfs/norasys/notebooks/camaret/model_checkpoints/nnint'}

In [22]:
from nnunetv2.utilities.label_handling.label_handling import determine_num_input_channels
from nnunetv2.utilities.find_class_by_name import recursive_find_python_class

from nnunetv2.utilities.plans_handling.plans_handler import PlansManager, ConfigurationManager
from batchgenerators.utilities.file_and_folder_operations import load_json, join, subdirs
from nnInteractive.trainer.nnInteractiveTrainer import nnInteractiveTrainer_stub

import nnInteractive
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_training_output_dir = model_path
dataset_json = load_json(join(model_training_output_dir, 'dataset.json'))
plans = load_json(os.path.join(model_training_output_dir, 'plans.json'))
plans_manager = PlansManager(plans)
checkpoint = torch.load(os.path.join(model_training_output_dir,"fold_0/checkpoint_final.pth"), map_location=device, weights_only=False)

configuration_name = checkpoint['init_args']['configuration']

parameters = checkpoint['network_weights']

configuration_manager = plans_manager.get_configuration(configuration_name)

trainer_name = checkpoint['trainer_name']

num_input_channels = determine_num_input_channels(plans_manager, configuration_manager, dataset_json)


trainer_class = recursive_find_python_class(join(nnInteractive.__path__[0], "trainer"),
                                            trainer_name, 'nnInteractive.trainer')
if trainer_class is None:
    print(f'Unable to locate trainer class {trainer_name} in nnInteractive.trainer. '
                        f'Please place it there (in any .py file)!')
    print('Attempting to use default nnInteractiveTrainer_stub. If you encounter errors, this is where you need to look!')
    trainer_class = nnInteractiveTrainer_stub

network = trainer_class.build_network_architecture(
    configuration_manager.network_arch_class_name,
    configuration_manager.network_arch_init_kwargs,
    configuration_manager.network_arch_init_kwargs_req_import,
    num_input_channels,
    plans_manager.get_label_manager(dataset_json).num_segmentation_heads,
    enable_deep_supervision=False
).to(device)
network.load_state_dict(parameters)





model_info = {
    "model_type": type(network).__name__,
    "model_module": type(network).__module__,
    "num_parameters": sum(p.numel() for p in network.parameters()),
    "input_channels": num_input_channels,
    "output_classes": plans_manager.get_label_manager(dataset_json).num_segmentation_heads,
    "configuration_name": configuration_name,
    "network_arch_class_name": configuration_manager.network_arch_class_name,
    "network_arch_init_kwargs": configuration_manager.network_arch_init_kwargs,
    "network_arch_init_kwargs_req_import": configuration_manager.network_arch_init_kwargs_req_import
}

print("Model Info:")
for key, value in model_info.items():
    print(f"{key}: {value}")
print("Model loaded successfully.")

RuntimeError: Error(s) in loading state_dict for ResidualEncoderUNet:
	size mismatch for encoder.stem.convs.0.conv.weight: copying a param with shape torch.Size([32, 1, 3, 3, 3]) from checkpoint, the shape in current model is torch.Size([32, 8, 3, 3, 3]).
	size mismatch for encoder.stem.convs.0.all_modules.0.weight: copying a param with shape torch.Size([32, 1, 3, 3, 3]) from checkpoint, the shape in current model is torch.Size([32, 8, 3, 3, 3]).
	size mismatch for decoder.encoder.stem.convs.0.conv.weight: copying a param with shape torch.Size([32, 1, 3, 3, 3]) from checkpoint, the shape in current model is torch.Size([32, 8, 3, 3, 3]).
	size mismatch for decoder.encoder.stem.convs.0.all_modules.0.weight: copying a param with shape torch.Size([32, 1, 3, 3, 3]) from checkpoint, the shape in current model is torch.Size([32, 8, 3, 3, 3]).

In [21]:
num_input_channels

1

In [23]:
num_input_channels

1